In [1]:
# !pip install -r ../../prosodic/requirements.txt
import sys
sys.path.insert(0,'..')
from generative_formalism import *
from generative_formalism.methods.memorization import *

In [2]:
df_smpl = get_genai_rhyme_completions_as_in_paper(by_line=True, filter_recognized=False)
df_smpl

* Loading legacy genai rhyme completions from {PATH_REPO}/data/corpus_genai_rhyme_completions.csv.gz
* Found 11298 unique human poems for input to models
* Found 21130 unique generated poems
* Distribution of input poem lengths
  6   [ 14 |    36 ]------                                              230
* Distribution of output poem lengths
  10  -- | 17 ]--                                                     100


100%|██████████| 326862/326862 [00:03<00:00, 87726.54it/s]


line_real  \
id_human                             model                    first_n_lines version date       id       stanza_num line_num                                                   
african-american/dunbarpa/Z200343481 claude-3-sonnet-20240229 5             1       2025-03-24 0f81663a 1          1                Long years ago, within a distant climb,   
                                                                                                                   2         Ere Love had touched me with his wand sublime,   
                                                                                                                   3            I dreamed of one to make my life's calm May   
                                                                                                                   4                 The panting passion of a summer's day.   
                                                                                                                   5                And ever since, in almost sad suspense,   
...                                                                                                                                                                     ...   
modern/sci0101/Z200480982            ollama/olmo2:latest      5             1       2025-03-15 9e7e696c 5          13                          "I enclose some photographs,   
                                                                                                                   14                               all done with a camera.   
                                                                                                        6          15          If they get crushed in the Post Office bags,   
                                                                                                                   16         they may easily be smoothed either by ironing   
                                                                                                                   17              or by being left some hours in a press."   

                                                                                                                                                              line_gen   line_sim  
id_human                             model                    first_n_lines version date       id       stanza_num line_num                                                        
african-american/dunbarpa/Z200343481 claude-3-sonnet-20240229 5             1       2025-03-24 0f81663a 1          1                                                          NaN  
                                                                                                                   2                                                          NaN  
                                                                                                                   3                                                          NaN  
                                                                                                                   4                                                          NaN  
                                                                                                                   5                                                          NaN  
...                                                                                                                                                                ...        ...  
modern/sci0101/Z200480982            ollama/olmo2:latest      5             1       2025-03-15 9e7e696c 5          13                  into lives interwoven by sight.  30.508475  
                                                                                                                   14        Now, with modern lenses, capturing light,  34.375000  
                                                                                      

In [5]:
get_memorized_poems_in_completions_as_in_paper(verbose=True)

100%|██████████| 326862/326862 [00:03<00:00, 93250.94it/s]


line_real  \
id_human                    model                          first_n_lines version date       id       stanza_num line_num                                            
english/miscell3/Z300440744 ollama/llama3.1:8b-text-q4_K_M 5             1       2025-03-23 001d9b46 1          1                 Tell me not, Sweet, I am unkind   
                                                                                                                2                           That from the nunnery   
                                                                                                                3            Of thy chaste breast and quiet mind,   
                                                                                                                4                          To war and arms I fly.   
                                                                                                     2          5               True, a new mistress now I chase,   
...                                                                                                                                                           ...   
american/am0539/Z200163743  ollama/llama3.1:8b-text-q4_K_M 5             1       2025-03-23 ff66a8b6 3          11            Kiss me softly and speak to me low;   
                                                                                                                12            Trust me, darling, the time is near   
                                                                                                                13            When we may love with never a fear;   
                                                                                                                14                                 Kiss me, dear!   
                                                                                                                15            Kiss me softly and speak to me low.   

                                                                                                                                                             line_gen    line_sim  
id_human                    model                          first_n_lines version date       id       stanza_num line_num                                                           
english/miscell3/Z300440744 ollama/llama3.1:8b-text-q4_K_M 5             1       2025-03-23 001d9b46 1          1                                                             NaN  
                                                                                                                2                                                             NaN  
                                                                                                                3                                                             NaN  
                                                                                                                4                                                             NaN  
                                                                                                     2          5                                                             NaN  
...                                                                                                                                                               ...         ...  
american/am0539/Z200163743  ollama/llama3.1:8b-text-q4_K_M 5             1       2025-03-23 ff66a8b6 3          11         If you want a life that is free from fear,   33.766234  
                                                                                                                12                    Keep a tight rein on your lips;   36.363636  
                                                                                                                13               Be careful what you say and how much   33.802817  
                          

## Looking for poems in training data

In [4]:
# !pip install prosodic
import pandas as pd
df = pd.read_pickle('./data.output.gen_poems.v3.pkl').reset_index()
df = df[df._model.str.contains('olmo2')]
ids = df._id.value_counts()
ids = ids[ids>=13].index.tolist()
len(ids), ids[:3]
import prosodic
import os
from wimbd.es import es_init, count_documents_containing_phrases
es = es_init(os.path.expanduser('~/Downloads/es_config_dolma_1_7_2.yml'))
from hashstash import HashStash
stash = HashStash('find_in_dolma', engine='pairtree', compress=False, b64=False)
# stash.clear()
df_meta=pd.read_csv(os.path.expanduser('~/lltk_data/corpora/chadwyck_poetry/metadata.csv')).set_index('id')

print('\n'.join(df[df._id=="english/brownieb/Z200289174"].line_real))
# df_meta.loc['c20-american/am22041/Z300232794'].dropna()
@stash.stashed_result
def find_in_dolma(id, index='docs_v1.7_2024-06-04'):
    dfg = df.query(f'_id=="{id}"')
    n = dfg.iloc[0]._first_n_lines
    dfg = dfg[dfg.line_num<=n]
    
    query = [ln.strip() for ln in dfg.sort_values('line_num').line_real]
    res = count_documents_containing_phrases(index, query, es=es, all_phrases=True)
    return {
        'id':id,
        'lines':query,
        'count':res
    }
# find_in_dolma('english/keachben/Z200407698')
ddf=find_in_dolma.stash.df.sort_values('count',ascending=False)
ddf['found']=ddf['count']>0
ddf[ddf['count']>0]
print(ddf.found.mean()*100)
ddf.head(25)

def find_all_in_dolma():
    from tqdm.auto import tqdm
    import random
    out = []
    random.shuffle(ids)
    iterr=tqdm(ids)
    for id in iterr:
        try:
            res = find_in_dolma(id)
            iterr.set_description(f'found {res["count"]} matches for {res["id"]}: {res["lines"][0]}')
            out.append(res)
        except Exception:
            pass
        # break
    odf = pd.DataFrame(out)
    return odf

FileNotFoundError: [Errno 2] No such file or directory: './data.output.gen_poems.v3.pkl'

In [ ]:
df1.columns

Index(['id', 'id_hash', 'period_meta', 'subcorpus', 'author', 'author_dob', 'title', 'year', 'num_lines', 'volume', 'line', 'rhyme', 'genre', 'period', 'txt'], dtype='object')

In [ ]:
df = get_rhyme_for_completed_poems(filter_line_sim=False)

100%|██████████| 22458/22458 [00:05<00:00, 3986.00it/s]


In [ ]:
df['found'] = df.line_sim > 95

In [ ]:
df_meta = get_chadwyck_corpus(period_by=50)
df_dolma=pd.read_pickle('/Users/ryan/Dropbox/Prof/Data/data.found_dolma.pkl')

# rename poem column to txt column
df_dolma = df_dolma.rename(columns={'poem':'txt'})
df_dolma_meta = df_dolma[['id','txt','count','found']].merge(df_meta, on='id')
df_dolma_meta.to_csv('../data/corpus_found_dolma.csv.gz', index=False)
df_dolma_meta

,id,txt,count,found,id_hash,period_meta,subcorpus,author,author_dob,title,year,num_lines,volume,line,rhyme,genre,period
0,c20-american/am20114/Z300221220,Tambourines!\nTambourines!\nTambourines\nTo th...,219,True,999655,1900-1999 Twentieth-Century,American Poetry,"Hughes, Langston, 1902-1967.",1902.0,Tambourines,1932,16,,Tambourines!,,,1900-1950
1,english/wattsisa/Z400522989,When I survey the wondrous cross\nOn which the...,99,True,515246,1750-1799 Later Eighteenth-Century,English Poetry,"Watts, Isaac, 1674-1748",1674.0,HYMN 7. (L. M.) Crucifixion to the World by th...,1704,20,The Works (1810),"On which the prince of glory dy'd,",y,Lyric,1650-1700
2,english-ed2/miscell3/Z300440750,"Go, lovely Rose!\nTell her, that wastes her ti...",97,True,218382,1550-1900 Miscellanies and Collections,English Poetry,"Waller, Edmund, 1606-1687",1606.0,"CXV [Go, lovely Rose!]",1636,20,,"&indent;Go, lovely Rose!",y,,1600-1650
3,c20-american/am20114/Z300220672,"I must say\nYes, sir,\nTo you all the time.\nY...",60,True,427210,1900-1999 Twentieth-Century,American Poetry,"Hughes, Langston, 1902-1967.",1902.0,Porter,1932,15,,I must say,,,1900-1950
4,english/wattsisa/Z400522722,Come let us join our cheerful songs\n With ...,31,True,325150,1750-1799 Later Eighteenth-Century,English Poetry,"Watts, Isaac, 1674-1748",1674.0,"HYMN 62. (C. M.) Christ Jesus, the Lamb of God...",1704,20,The Works (1810),&indent;With angels round the throne;,y,Lyric,1650-1700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4630,c20-african-american/da20096/Z200331275,Off go the crows from the roof.\nThe crows can...,0,False,673433,1900-1999 Twentieth-Century,African-American Poetry,"Eady, Cornelius, 1954-",1954.0,CROWS IN A STRONG WIND,1984,20,,Off go the crows from the roof.,,,1950-2000
4631,c20-american/am30242/Z300570450,The trees in clouds of November missed\nStandi...,0,False,154819,1900-1999 Twentieth-Century,American Poetry,"Kinnell, Galway, 1927-",1927.0,EARTH&hyphen;SPARROW,1957,17,,The trees in clouds of November mist,,,1900-1950
4632,english/priormat/Z300465735,"Haste my Nannette, my lovely maid,\nHaste to t...",0,False,533684,1700-1749 Early Eighteenth-Century,English Poetry,"Prior, Matthew, 1664-1721",1664.0,Nannette. A Song.,1694,14,Dialogues of the Dead (1907),"Haste to the bower, thy swain has made.",y,,1650-1700
4633,american/am0076/Z200145561,Thine eyes shall see the light of distant skie...,0,False,657643,1835-1869 Mid Nineteenth-Century,American Poetry,"Bryant, William Cullen, 1794-1878",1794.0,"TO COLE, THE PAINTER, DEPARTING FOR EUROPE.",1824,14,The Poetical Works (1903),"&indent;A living image of our own bright land,",y,Sonnet,1750-1800


In [ ]:
def get_antoniak_et_al_memorization_data(poetry_eval_repo_dir='../../poetry-eval'):
    data_fldr = os.path.join(poetry_eval_repo_dir, 'data')
    df_antoniak = pd.read_csv(os.path.join(data_fldr, 'poetry-evaluation_public-domain-poems.csv'))
    df_antoniak['id'] = df_antoniak['poem_link'].apply(lambda x: '/'.join(x.split('/')[-2:]))
    df_antoniak = df_antoniak.rename(columns={'poem_text':'txt'})
    df_antoniak = df_antoniak.set_index('id')
    # df_walsh.to_csv('../data/antoniak_et_al.poetry-evaluation_public-domain-poems.csv.gz')

    df_mem=pd.read_csv(os.path.join(data_fldr, 'memorization_results.csv')).drop('Unnamed: 0',axis=1)
    df_mem = df_mem.rename(columns={'poem':'id', 'prediction':'found_closed'})
    df_mem['found_closed'] = df_mem['found_closed'].astype(bool)
    df_mem_antoniak = df_antoniak.merge(df_mem, on='id').set_index('id')
    
    df_mem_antoniak['found_open'] = False
    
    for fn in os.listdir(data_fldr):
        if 'wimbd' in fn:
            fndf=pd.read_csv(os.path.join(data_fldr,fn))
            for poem_id in fndf.poem_id.unique():
                if poem_id in df_mem_antoniak.index:
                    df_mem_antoniak.loc[poem_id, 'found_open'] = True
    
    df_mem_antoniak.to_csv('../data/antoniak_et_al_memorization_results.csv.gz')
    return df_mem_antoniak



In [ ]:
df_antoniak_mem = get_antoniak_et_al_memorization_data()
df_antoniak_mem

,author,birth_death_dates,poem_title,txt,form,form_group,tags,poem_source,poem_link,author_link,pub_year,extracted_birth_year,extracted_death_year,form_tags,theme_tags,occasion_tags,collected_from,found_closed,found_open
id,,,,,,,,,,,,,,,,,,,
43926/the-canterbury-tales-general-prologue,Geoffrey Chaucer,?–1400,The Canterbury Tales: General Prologue,Here bygynneth the Book of the tales of Caunte...,couplet,stanza forms,"['The Body', 'The Mind', 'Love', 'Activities',...",Unknown,https://www.poetryfoundation.org/poems/43926/t...,https://www.poetryfoundation.org/poets/geoffre...,NaN,NaN,1400.0,[],[],[],Poetry Foundation,True,True
144612/where-did-the-handsome-beloved-go,Jalal al-Din Rumi,1207–1273,“Where did the handsome beloved go?”,"Where did the handsome beloved go?\nI wonder, ...",ghazal,verse forms,"['Appeared in Poetry Magazine', 'Living', 'Mar...",Source:\n Poetry\n ...,https://www.poetryfoundation.org/poetrymagazin...,https://www.poetryfoundation.org/poets/jalal-a...,NaN,1207.0,1273.0,[],[],[],Poetry Foundation,False,False
poem/sonnet-131-id-sing-love-such-novel-fashion,Petrarch,1304 – 1374,Sonnet 131 [I’d sing of Love in such a novel f...,I’d sing of Love in such a novel fashion\nthat...,sonnet,verse forms,"['Sonnet', 'Fathers', 'Translation']","From The Poetry of Petrarch by Petrarch, trans...",https://poets.org/poem/sonnet-131-id-sing-love...,https://poets.org/poet/petrarch,2004.0,1304.0,1374.0,['Sonnet'],"['Fathers', 'Translation']",[],Academy of American Poets,False,True
poem/sonnet-102-if-no-love-o-god-what-fele-i-so,Petrarch,1304 – 1374,"Sonnet 102 [If no love is, O God, what fele I ...","If no love is, O God, what fele I so?\n And...",sonnet,verse forms,"['Sonnet', 'Heartache', 'Public Domain', 'Tran...","From Troilus and Criseyde, as translated by G...",https://poets.org/poem/sonnet-102-if-no-love-o...,https://poets.org/poet/petrarch,1375.0,1304.0,1374.0,['Sonnet'],"['Heartache', 'Public Domain', 'Translation']",[],Academy of American Poets,True,True
poem/sonnet-101-ways-apt-and-new-sing-love-id-find,Petrarch,1304 – 1374,Sonnet 101 [Ways apt and new to sing of love I...,"Ways apt and new to sing of love I’d find,\nFo...",sonnet,verse forms,"['Sonnet', 'Public Domain']",Translated by Robert Guthrie Macgregor. This p...,https://poets.org/poem/sonnet-101-ways-apt-and...,https://poets.org/poet/petrarch,1374.0,1304.0,1374.0,['Sonnet'],['Public Domain'],[],Academy of American Poets,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
poem/unquiet-grave,Anonymous,NaN,The Unquiet Grave,"I\n\n'The wind doth blow today, my love,\n An...",ballad,verse forms,"['Ballad', 'Public Domain']",This poem is in the public domain.,https://poets.org/poem/unquiet-grave,https://poets.org/poet/anonymous,1400.0,NaN,NaN,['Ballad'],['Public Domain'],[],Academy of American Poets,True,True
poem/sonnet-7-soote-season-bud-and-bloom-forth-brings,Henry Howard,NaN,"Sonnet 7 [The soote season, that bud and bloom...","The soote season, that bud and bloom forth bri...",sonnet,verse forms,"['Sonnet', 'Public Domain', 'Summer']",This poem is in the public domain.,https://poets.org/poem/sonnet-7-soote-season-b...,https://poets.org/poet/henry-howard,1374.0,NaN,NaN,['Sonnet'],['Public Domain'],['Summer'],Academy of American Poets,True,False
50399/earth-upon-earth,Anonymous,NaN,Earth Upon Earth,"Earth took of earth, earth with woe,\nEarth ot...",free verse,meters,"['Nature', 'Free Verse']",Unknown,https://www.poetryfoundation.org/poems/50399/e...,NaN,NaN,NaN,NaN,[],[],[],Poetry Foundation,True,False


found_open
False    865
True     771
Name: count, dtype: int64

In [ ]:
df_mem=pd.read_csv('../../poetry-eval/data/memorization_results.csv').drop('Unnamed: 0',axis=1)
df_mem = df_mem.rename(columns={'poem':'poem_id', 'prediction':'found'})
df_mem['found_closed'] = df_mem['found'].astype(bool)
df_mem_walsh = df_mem.merge(df_walsh, on='poem_id').set_index('poem_id')
df_mem_walsh

In [ ]:
df_walsh2 = pd.concat([
    df_mem_walsh.assign(found_source='Memorized by closed models (Walsh et al)'),
    odf_walsh2.assign(found_source='Found in open model training data (Walsh et al)'),
])
df_walsh2

In [ ]:
figdf5 = pd.concat([
    figdf4,
    df_walsh2
])
figdf5.groupby(['found_source','found']).size()

In [ ]:
import plotnine as p9
p9.options.figure_size=8,6
p9.options.dpi=300
fig = p9.ggplot(figdf5, p9.aes('perc_rhyming_lines', color='found', fill='found'))
fig += p9.geom_density(alpha=.1)
fig+= p9.theme_minimal()
fig += p9.facet_wrap('found_source')
fig+= p9.theme(
    legend_position='bottom',
    plot_background=p9.element_rect(fill='white', color=None),  # Add white background
    panel_background=p9.element_rect(fill='white', color=None)  # Add white background to panels
)
fig+= p9.labs(
    y='Probability density',
    x='Percent of lines rhyming',
    color='Poem in LLM training data',
    fill='Poem in LLM training data',
    title='Poems in LLM training data are not disproportionately rhyming',
    # caption='Note: Given the first 5 lines of 10-20 line poems from poets born in each century, 1600-2000, LLMs are prompted to "complete" the poem.\nRhyme is measured by exact phoneme match in the rime of the final syllable (or syllables, if final syllable unstressed).\nPoems randomly sampled from Chadwyck-Healey poetry collections, with 600 poems for each model for each century.\nResults shown for actual poems as well as the LLM imitations. Poems "memorized" by the model are excluded.'
)
fig.save(os.path.expanduser('~/Dropbox/Prof/Data/fig.rhyme_distros_by_memorized2.png'))
fig